In [1]:
#@title Run on TensorFlow 2.x
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
#@title Import relevant modules
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from mlp_sparse_model import MLPSparseModel
from mlp_plain_model import MLPPlainModel
import time

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.7f}".format

# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)

SAMPLE_SIZE = 11
N_EXP = 20

In [3]:
def seed_generator():
    # Generate the initial seed for each sample size (to match the seed
    # of the results in the paper)
    # This is just the initial seed, for each experiment, the seeds will be
    # equal the initial seed + the number of the experiment

    N_train_all = np.multiply(11, [1, 2, 3, 4, 5])  # This is for LLVM
    if SAMPLE_SIZE in N_train_all:
        seed_o = np.where(N_train_all == SAMPLE_SIZE)[0][0]
    else:
        seed_o = np.random.randint(1, 101)

    return seed_o

In [4]:
#@title Get data
data_df = pd.read_csv("Data/LLVM_AllNumeric.csv")
column_dict = {name: bool for name in list(data_df.columns.values) if name != 'PERF'}
column_dict['PERF'] = "float64"
data_df = data_df.astype(column_dict)
data_df = data_df.reindex(np.random.permutation(data_df.index))

In [9]:
# split data set and set seed
seed_init = seed_generator()
seed = seed_init*N_EXP + 1
np.random.seed(seed_init)
train_df, validate_df, test_df = np.split(data_df.sample(frac=1), [int(.5*len(data_df)), int(.9*len(data_df))])
display()